<a href="https://colab.research.google.com/github/eunzizizip/2025_AI/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, cv2, random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split



In [3]:
data_dir = "/content/drive/MyDrive/Vehicle_dataset/Vehicles"
categories = ['Auto Rickshaws', 'Bikes', 'Cars', 'Motorcycles', 'Planes', 'Ships', 'Trains']
data = []

for category in categories:
    path = os.path.join(data_dir, category)
    print(f"현재 처리 중인 폴더: {path}")

    class_num = categories.index(category)

    for img_name in os.listdir(path):
        try:
            if img_name.endswith(".jpg"):
                img_path = os.path.join(path, img_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (64, 64))
                data.append([img, class_num])
        except Exception as e:
            print(f"이미지 처리 중 오류 발생: {img_name} / {e}")

# 데이터 섞기
random.shuffle(data)
print(f"총 이미지 개수: {len(data)}")


현재 처리 중인 폴더: /content/drive/MyDrive/Vehicle_dataset/Vehicles/Auto Rickshaws
현재 처리 중인 폴더: /content/drive/MyDrive/Vehicle_dataset/Vehicles/Bikes
현재 처리 중인 폴더: /content/drive/MyDrive/Vehicle_dataset/Vehicles/Cars
현재 처리 중인 폴더: /content/drive/MyDrive/Vehicle_dataset/Vehicles/Motorcycles
현재 처리 중인 폴더: /content/drive/MyDrive/Vehicle_dataset/Vehicles/Planes
현재 처리 중인 폴더: /content/drive/MyDrive/Vehicle_dataset/Vehicles/Ships
현재 처리 중인 폴더: /content/drive/MyDrive/Vehicle_dataset/Vehicles/Trains
총 이미지 개수: 5007


In [4]:

# 이미지와 레이블 분리
X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

print(f"총 이미지 수: {len(X)}")
print(f"총 라벨 수: {len(y)}")

# NumPy 배열로 변환하고 정규화
X = np.array(X).reshape(-1, 64, 64, 3) / 255.0
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("y 라벨 예시:", y[:10]) #10개만

# 레이블을 원-핫 인코딩
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)

# 학습/검증용 데이터로 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("학습용 데이터 크기:", x_train.shape)
print("테스트용 데이터 크기:", x_test.shape)


총 이미지 수: 5007
총 라벨 수: 5007
X shape: (5007, 64, 64, 3)
y shape: (5007,)
y 라벨 예시: [0 4 1 3 4 5 1 1 3 3]
학습용 데이터 크기: (4005, 64, 64, 3)
테스트용 데이터 크기: (1002, 64, 64, 3)


In [ ]:
# 모델 구성
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7개 클래스
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,626,055 (6.20 MB)

 Trainable params: 1,626,055 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 모델 컴파일 및 훈련
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 28s 195ms/step - accuracy: 0.3206 - loss: 1.7460 - val_accuracy: 0.6307 - val_loss: 1.0452
Epoch 2/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 40s 191ms/step - accuracy: 0.6445 - loss: 1.0364 - val_accuracy: 0.7335 - val_loss: 0.8182
Epoch 3/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 42s 199ms/step - accuracy: 0.7250 - loss: 0.8159 - val_accuracy: 0.7774 - val_loss: 0.6803
Epoch 4/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 22s 172ms/step - accuracy: 0.7835 - loss: 0.6538 - val_accuracy: 0.7625 - val_loss: 0.6738
Epoch 5/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 41s 173ms/step - accuracy: 0.7949 - loss: 0.5921 - val_accuracy: 0.7834 - val_loss: 0.6468
Epoch 6/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 43s 188ms/step - accuracy: 0.8380 - loss: 0.4886 - val_accuracy: 0.8174 - val_loss: 0.5513
Epoch 7/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 23s 181ms/step - accuracy: 0.8587 - loss: 0.4156 - val_accuracy: 0.8204 - val_loss: 0.5517
Epoch 8/30
126/126 ━━━━━━━━━━━━━━━━━━━━ 44s 205ms/step - accuracy: 0.8771 - loss: 0

In [ ]:
import numpy as np
import cv2
import os

def prepare_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (64, 64))
    img = np.array(img) / 255.0
    return img

data_dir = "/content/drive/MyDrive/Vehicle_dataset/Vehicles"
test_categories = ['Auto Rickshaws', 'Bikes', 'Cars', 'Motorcycles', 'Planes', 'Ships', 'Trains']

batch_size = 64
correct = 0
total = 0

for idx, category in enumerate(test_categories):
    category_path = os.path.join(data_dir, category)
    image_paths = [os.path.join(category_path, f) for f in os.listdir(category_path) if f.endswith(".jpg")]

    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i + batch_size]
        batch_images = [prepare_image(p) for p in batch_paths]
        batch_input = np.array(batch_images).reshape(-1, 64, 64, 3)

        predictions = model.predict(batch_input)
        predicted_labels = np.argmax(predictions, axis=1)

        for pred_label in predicted_labels:
            if pred_label == idx:
                correct += 1
            total += 1

print(f"정확도: {correct}/{total} = {correct / total:.2%}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
2/2 ━━